<a href="https://colab.research.google.com/github/talgalper/Honours-2021/blob/main/wilcoxon_stat_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install ncbi-blast+
!apt-get install exonerate
!apt-get install dssp
!pip install biostructmap

In [2]:
import pandas as pd
from collections import defaultdict
import biostructmap

/usr/local/lib/python3.7/dist-packages/Bio/SubsMat/__init__.py:131: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  BiopythonDeprecationWarning,


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
#read in data file and file in gaps with 0 values
def nextstrain_data_processing(filename, seq_len, gene_id):
    ns_data = pd.read_csv(filename, sep='\t')

    s_gene_data = ns_data[ns_data.gene == gene_id]

    pos = s_gene_data.position

    entropy = s_gene_data.entropy

    pos_to_entropy = defaultdict(float, zip(pos, entropy))

    len_of_ref_seq = seq_len

    list_of_values = [pos_to_entropy[i+1] for i in range(len_of_ref_seq)]
    return list_of_values

In [5]:
s_protein_data = nextstrain_data_processing('/content/drive/MyDrive/Honours/nextstrain_ncov_gisaid_global_diversity_Jan2020-Aug2021.tsv', 1273, 'S')

In [6]:
#removes fasta file header
def ref_seq_fasta_format(fasta_filename):
    with open(fasta_filename) as f:
        seq_lines = f.readlines()
    
    formatted_seq_lines = seq_lines[1:]
    
    new_list_lines_removed = list(map(str.strip, formatted_seq_lines)) 
    complete_format = ''.join(new_list_lines_removed)
    return complete_format

In [7]:
ref_seq = ref_seq_fasta_format('/content/drive/MyDrive/Honours/Reference_Sequence/sars_cov_2_spike_ref_seq.fasta')

In [8]:
#check
ref_seq

'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITG

In [9]:
#Biostruct map initalisation
def biostructmap_analysis(pdb_structure, gene_name, input_data, ref_sequence, output_filename):
    structure = biostructmap.Structure(pdb_structure, gene_name)

    data = {'A': input_data, 
            'B': input_data, 
            'C': input_data
            }
 
    ref_seqs = {'A': ref_sequence,
                'B': ref_sequence,
                'C': ref_sequence
                }

    results = structure.map(data, method='default', ref=ref_seqs, radius=15, rsa_range=(0.2, 1.0))

    results.write_data_to_pdb_b_factor(fileobj=output_filename, scale_factor=100)
    return results  # Note we are now returning the results object as well as saving data to a file. 

In [10]:
mapped_diversity_data = biostructmap_analysis('/content/drive/MyDrive/Honours/Reference_Sequence/spike_swiss_model.pdb', 'spike_protein', s_protein_data, ref_seq, 'biostructmap_diversity_Jan2020-Aug2021.pdb')

In [11]:
# Get data for a particular residue:
chain_of_interest = 'A'
residue_id_of_interest = 20

# Need to use Biopython's slightly odd residue identifier syntax.
residue_of_interest = (chain_of_interest, (' ', residue_id_of_interest, ' '))
mapped_diversity_data[residue_of_interest]

0.07741935483870968

In [ ]:
# How you might go about getting data out for all months:

month_data_files = ['/content/drive/MyDrive/Honours/Monthly NextStrain data/02nextstrain_ncov_global_diversity_Jan_2020.tsv', 
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/03nextstrain_ncov_global_diversity_Feb_2020.tsv', 
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/04nextstrain_ncov_global_diversity_Mar_2020.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/05nextstrain_ncov_global_diversity_Apr_2020.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/06nextstrain_ncov_global_diversity_May_2020.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/07nextstrain_ncov_global_diversity_Jun_2020.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/08nextstrain_ncov_global_diversity_Jul_2020.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/09nextstrain_ncov_global_diversity_Aug_2020.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/10nextstrain_ncov_global_diversity_Sep_2020.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/11nextstrain_ncov_global_diversity_Oct_2020.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/12nextstrain_ncov_global_diversity_Nov_2020.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/13nextstrain_ncov_global_diversity_Dec_2020.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/14nextstrain_ncov_global_diversity_Jan_2021.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/15nextstrain_ncov_global_diversity_Feb_2021.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/16nextstrain_ncov_global_diversity_Mar_2021.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/17nextstrain_ncov_global_diversity_Apr_2021.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/18nextstrain_ncov_global_diversity_May_2021.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/19nextstrain_ncov_global_diversity_Jun_2021.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/20nextstrain_ncov_gisaid_global_diversity_Jul_2021.tsv',
                    '/content/drive/MyDrive/Honours/Monthly NextStrain data/21nextstrain_ncov_global_diversity_Aug_2021.tsv']
month_biostructmap_data = []

# Simply looping over all month files, performing analysis, saving results in a list.
for month_data in month_data_files:
    month_file_prefix = month_data.replace('.tsv', '')
    s_protein_data = nextstrain_data_processing(month_data, 1273, 'S')
    mapped_diversity_data = biostructmap_analysis('/content/drive/MyDrive/Honours/Reference_Sequence/spike_swiss_model.pdb', 'spike_protein', s_protein_data, ref_seq, f"{month_file_prefix}_biostructmap.pdb")
    month_biostructmap_data.append(mapped_diversity_data)

In [ ]:
# And this is how you might go about getting data out for a particular residue across all months
# Get data for a particular residue:
chain_of_interest = 'A'
residue_id_of_interest = 144

# Need to use Biopython's slightly odd residue identifier syntax.
residue_of_interest = (chain_of_interest, (' ', residue_id_of_interest, ' '))
data_for_residue = [x[residue_of_interest] for x in month_biostructmap_data]

data_for_residue

# And now you can plot this using the `data_for_residue` list as your y values, and the months as your x values.

In [ ]:
# Example of how you could pull out values across all months to then calculate stats on:

residue_div_values_per_month = []
chain_of_interest = 'A' #Only calculate on a single chain

for single_month_data in month_biostructmap_data:
    diversity_values = [value for key, value in single_month_data.items() if key[0] == chain_of_interest]
    residue_div_values_per_month.append(diversity_values)

from scipy.stats import wilcoxon
# Now you can use the `residue_div_values_per_month` list to do stats comparing months.

for i, month_1 in enumerate(month_biostructmap_data):
    for j, month_2 in enumerate(month_biostructmap_data):
        stat, p = wilcoxon(month_1, month_2)
        print(f"P value for comparison between months {i} and {j} is {p}\n")


        # Note there is a bit more work needed here, for example you might need to create a matrix plot, or add a lookup list with properly formatted dates so you know what each month index is referring to.
        # But this will get you started.